### Finding Prime Numbers

In [1]:
from pyspark import SparkContext
sc = SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
n = 500000
allnumbers = sc.parallelize(range(2, n), 8).cache()
composite = allnumbers.flatMap(lambda x: range(x*2, n, x))
# composite = allnumbers.flatMap(lambda x: range(x*2, n, x)).repartition(8) 
# This can rebalance the partition, improve the speed of last partition
prime = allnumbers.subtract(composite)
print (prime.take(10))

[17, 97, 113, 193, 241, 257, 337, 353, 401, 433]


In [11]:
# Find the number of elements in each parttion
def partitionsize(it): 
    #s = 0
    #for i in it:
    #    s += 1
    #yield s
    yield len(list(it)) # 这行代码等价于上面的代码

print (allnumbers.mapPartitions(partitionsize).collect())
print (composite.mapPartitions(partitionsize).collect())
print (prime.mapPartitions(partitionsize).collect())
print (prime.glom().collect()[1][0:4])

[62499, 62500, 62500, 62500, 62499, 62500, 62500, 62500]
[5216986, 254759, 104166, 62499, 0, 0, 0, 0]
[0, 5169, 1, 5219, 0, 5206, 0, 5189, 0, 5165, 0, 5199, 0, 5191, 0, 5199]
[17, 97, 113, 193]


### Data Partitioning

In [18]:
data = [8, 96, 240, 400, 1, 800]
rdd = sc.parallelize(zip(data, data),4)
print (rdd.partitioner) # 虽然设置了partition = 4，但实际上却并没有partition
print (rdd.glom().collect())

rdd = rdd.reduceByKey(lambda x,y: x+y)
print (rdd.glom().collect())
print (rdd.partitioner.partitionFunc) # 使用reduceByKey后，居然partition了

rdd1 = rdd.mapValues(lambda x: x+1)
print (rdd1.glom().collect())
print (rdd1.partitioner.partitionFunc)

rdd = rdd.sortByKey() # 使用sortByKey后，不仅partition了，而且partition还是balance的
print (rdd.glom().collect())
print (rdd.partitioner.partitionFunc)

None
[[(8, 8)], [(96, 96), (240, 240)], [(400, 400)], [(1, 1), (800, 800)]]
[[(8, 8), (96, 96), (240, 240), (400, 400), (800, 800)], [(1, 1)], [], []]
<function portable_hash at 0x7f89705f3378>
[[(8, 9), (96, 97), (240, 241), (400, 401), (800, 801)], [(1, 2)], [], []]
<function portable_hash at 0x7f89705f3378>
[[(1, 1), (8, 8)], [(96, 96), (240, 240)], [(400, 400)], [(800, 800)]]
<function RDD.sortByKey.<locals>.rangePartitioner at 0x7f894c1d9c80>


In [22]:
def partitionsize(it): yield len(list(it))
    
n = 40000

def f(x):
    return x % 9

data1 = list(range(0, n, 16)) + list(range(0, n, 16))
data2 = range(0, n, 8)
rdd1 = sc.parallelize(zip(data1, data2), 8)
print (rdd1.mapPartitions(partitionsize).collect())
rdd2 = rdd1.reduceByKey(lambda x,y: x+y)
print (rdd2.mapPartitions(partitionsize).collect())
rdd3 = rdd2.partitionBy(8, f)
print (rdd3.mapPartitions(partitionsize).collect())
rdd4 = rdd1.reduceByKey(lambda x,y: x+y, partitionFunc=f)
print (rdd4.mapPartitions(partitionsize).collect())

[625, 625, 625, 625, 625, 625, 625, 625]
[2500, 0, 0, 0, 0, 0, 0, 0]
[556, 278, 277, 278, 277, 278, 278, 278]
[556, 278, 277, 278, 277, 278, 278, 278]


In [24]:
a = sc.parallelize(zip(range(10000), range(10000)), 8)
b = sc.parallelize(zip(range(10000), range(10000)), 8)
print (a.partitioner)
a = a.reduceByKey(lambda x,y: x+y)
print (a.partitioner.partitionFunc)
b = b.reduceByKey(lambda x,y: x+y) # (1)
c = a.join(b)  
# 不需要shuffle operation，因为上一行代码b.reduceByKey(lambda x,y: x+y)
# 会让spark认为出a和b是一样的
print (c.getNumPartitions()) # 去掉(1)代码，结果为16；保留(1)，结果为8
print (c.partitioner.partitionFunc)
print (c.glom().first()[0:4])

None
<function portable_hash at 0x7f89705f3378>
16
<function portable_hash at 0x7f89705f3378>
[(0, (0, 0)), (16, (16, 16)), (32, (32, 32)), (48, (48, 48))]


### Partitioning in DataFrames

In [26]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
spark

In [27]:
data1 = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4]
data2 = [2, 2, 3, 4, 5, 3, 1, 1, 2, 3]
df = spark.createDataFrame(zip(data1, data2))
print (df.rdd.getNumPartitions())
print (df.rdd.glom().collect())

4
[[Row(_1=1, _2=2), Row(_1=1, _2=2)], [Row(_1=1, _2=3), Row(_1=2, _2=4)], [Row(_1=2, _2=5), Row(_1=2, _2=3)], [Row(_1=3, _2=1), Row(_1=3, _2=1), Row(_1=3, _2=2), Row(_1=4, _2=3)]]


In [29]:
df1 = df.repartition(6, df._1)
print (df1.rdd.glom().collect())
df1.show()

[[], [], [Row(_1=2, _2=4), Row(_1=2, _2=5), Row(_1=2, _2=3), Row(_1=4, _2=3)], [Row(_1=3, _2=1), Row(_1=3, _2=1), Row(_1=3, _2=2)], [], [Row(_1=1, _2=2), Row(_1=1, _2=2), Row(_1=1, _2=3)]]
+---+---+
| _1| _2|
+---+---+
|  2|  4|
|  2|  5|
|  2|  3|
|  4|  3|
|  3|  1|
|  3|  1|
|  3|  2|
|  1|  2|
|  1|  2|
|  1|  3|
+---+---+



In [31]:
# A 'real' example from SF Express
# Prepare three relational tables

from pyspark.sql.functions import *

num_waybills = 1000
num_customers = 100

# floor向上取整
rdd = sc.parallelize((i, ) for i in range(num_waybills))
waybills = spark.createDataFrame(rdd).select(floor(rand()*num_waybills).alias('waybill'), 
                                             floor(rand()*num_customers).alias('customer')) \
                .repartition('waybill')\
                .cache()
waybills.show()
print (waybills.count())

rdd = sc.parallelize((i, i) for i in range(num_customers))
customers = spark.createDataFrame(rdd, ['customer', 'phone']).cache()
customers.show()
print (customers.count())

rdd = sc.parallelize((i, ) for i in range(num_waybills))
waybill_status = spark.createDataFrame(rdd).select(floor(rand()*num_waybills).alias('waybill'), 
                                                   floor(rand()*10).alias('version')) \
                      .groupBy('waybill').max('version').cache()
waybill_status.show()
print (waybill_status.count())

+-------+--------+
|waybill|customer|
+-------+--------+
|     26|      87|
|     29|       9|
|     26|      15|
|    474|       6|
|    474|      50|
|    964|      77|
|    558|      17|
|    541|      11|
|    558|       5|
|     65|       3|
|    270|       7|
|    270|      95|
|    270|      73|
|    270|      87|
|    730|      20|
|    278|      91|
|    705|      51|
|    278|      32|
|    278|       3|
|     19|      78|
+-------+--------+
only showing top 20 rows

1000
+--------+-----+
|customer|phone|
+--------+-----+
|       0|    0|
|       1|    1|
|       2|    2|
|       3|    3|
|       4|    4|
|       5|    5|
|       6|    6|
|       7|    7|
|       8|    8|
|       9|    9|
|      10|   10|
|      11|   11|
|      12|   12|
|      13|   13|
|      14|   14|
|      15|   15|
|      16|   16|
|      17|   17|
|      18|   18|
|      19|   19|
+--------+-----+
only showing top 20 rows

100
+-------+------------+
|waybill|max(version)|
+-------+------------+
|     

In [32]:
# We want to join 3 tables together.
# Knowing how each table is partitioned helps optimize the join order.

waybills.join(customers, 'customer').join(waybill_status, 'waybill').show()
# waybills.join(waybill_status, 'waybill').join(customers, 'customer').show()

+-------+--------+-----+------------+
|waybill|customer|phone|max(version)|
+-------+--------+-----+------------+
|     26|      87|   87|           3|
|     29|       9|    9|           7|
|     26|      15|   15|           3|
|    474|       6|    6|           9|
|    474|      50|   50|           9|
|    964|      77|   77|           1|
|    558|      17|   17|           7|
|    541|      11|   11|           0|
|    558|       5|    5|           7|
|     65|       3|    3|           4|
|    270|       7|    7|           8|
|    270|      95|   95|           8|
|    270|      73|   73|           8|
|    270|      87|   87|           8|
|    705|      51|   51|           8|
|     19|      78|   78|           9|
|    926|      28|   28|           5|
|    926|      75|   75|           5|
|    926|       3|    3|           5|
|    926|      95|   95|           5|
+-------+--------+-----+------------+
only showing top 20 rows



### Threading

In [36]:
import threading
import random

partitions = 20
# partitions = 1 
# 1----
# 2----
# 3----
# 4----
# 5----
# partition = 20
# 1----
# 2----
# 3---------
# 4-----------
# 5---------------
# This is because 3,4,5 workers wait the 1,2 finishing their jobs and release the cores for computing

n = 500000 * partitions

# use different seeds in different threads and different partitions
# a bit ugly, since mapPartitionsWithIndex takes a function with only index
# and it as parameters
def f1(index, it):
    random.seed(index + 987231)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

def f2(index, it):
    random.seed(index + 987232)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

def f3(index, it):
    random.seed(index + 987233)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0
    
def f4(index, it):
    random.seed(index + 987234)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0
    
def f5(index, it):
    random.seed(index + 987245)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

f = [f1, f2, f3, f4, f5]
    
# the function executed in each thread/job
def dojob(i):
    count = sc.parallelize(range(1, n + 1), partitions) \
              .mapPartitionsWithIndex(f[i]).reduce(lambda a,b: a+b)
    print ("Worker", i, "reports: Pi is roughly", 4.0 * count / n)

# create and execute the threads
threads = []
for i in range(5):
    t = threading.Thread(target=dojob, args=(i,))
    threads += [t]
    t.start()

# wait for all threads to complete
for t in threads:
    t.join()    
'''
for i in range(5):
    dojob(i)
'''

Worker 3 reports: Pi is roughly 3.1420744
Worker 2 reports: Pi is roughly 3.1421896
Worker 0 reports: Pi is roughly 3.1422884
Worker 1 reports: Pi is roughly 3.1422096
Worker 4 reports: Pi is roughly 3.1416516


'\nfor i in range(5):\n    dojob(i)\n'